## Comandos SQL ejecutados dentro de código Python usando el módulo SQLite3.

In [1]:
import sqlite3

Para crear la base de datos como punto de inicio, el equivalente en SQL a:
* CREATE DATABASE dataBaseName;

Sería conn = sqlite3.connect("Samsung_Tecs_Projects_MAD.sqlite3")

* USE dataBaseName;

Sería c = conn.cursor()

In [2]:
conn = sqlite3.connect("Samsung_Tecs_Projects_MAD.sqlite3")
c = conn.cursor()

El resto de comandos SQL (en mayúsculas) se ejecutan dentro de:

`c.execute("""
comandos SQL a ejecutar ;
""")`

## Creating Tables

**Creating table 1:** project_phases

In [3]:
c.execute("""
CREATE TABLE project_phases(
Phase_id INTEGER PRIMARY KEY,
Phase_name TEXT UNIQUE
);
""")

**Creating table 2:** project_identification

In [4]:
c.execute("""
CREATE TABLE project_identification(
Project_id INTEGER AUTO INCREMENT,
Project_Name TEXT,
Phase_id INTEGER,
Related_Projects BLOB NOT NULL,
Describe TEXT NOT NULL,
FOREIGN KEY (Phase_id) REFERENCES project_phases(Phase_id)
);
""")

**Creating table 3:** project_people

In [5]:
c.execute("""
CREATE TABLE project_people(
Employee_id TEXT NOT NULL UNIQUE,
F_Name TEXT NOT NULL,
L_Name TEXT NOT NULL,
email TEXT NOT NULL,
Phone_num TEXT DEFAULT 'no info.',
P_manager BLOB,
P_developer BLOB DEFAULT 0,
Currently_member_of_project_a INTEGER, 
Currently_member_of_project_b INTEGER, 
Currently_member_of_project_c INTEGER, 
FOREIGN KEY (Currently_member_of_project_a) REFERENCES project_identification(Project_id),
FOREIGN KEY (Currently_member_of_project_b) REFERENCES project_identification(Project_id),
FOREIGN KEY (Currently_member_of_project_c) REFERENCES project_identification(Project_id)
);
""")

## Inserting some records

**INTO project_phases**

In [6]:
c.execute("""
INSERT INTO project_phases
VALUES
(0, 'Stand-by / Paused'),
(1, 'Planning / Defining'),
(2, 'Designing'),
(3, 'Developing'),
(4, 'Testing'),
(5, 'Go-live'),
(6, 'Follow-up / Maintenance'),
(7, 'Cancelled'),
(8, 'Ended / Closed')
;
""")

**INTO project_identification**

In [7]:
c.execute("""
INSERT INTO project_identification
VALUES
(0, 'no project assigned', 0, 0, 'none'),
(1, 'Fibra óptica Paracuellos de Jarama', 1, 0, 'none'),
(2, 'Reestructuración BBDD de Mercedes-Benz Alcobendas', 4, 0, 'none'),
(3, 'Canales de comunicación del Ayuntamiento SS.Reyes', 6, 0, 'none'),
(4, 'Fibra óptica Ayto SS.Reyes', 1, 1, 'Relacionado con proyecto id 3.'),
(5, 'Desarrollo de ventana de búsqueda para Dpto Comercial', 3, 0, 'Interno')
;
""")

**INTO project_people**

In [8]:
c.execute("""
INSERT INTO project_people
VALUES
('N-MAD-000001', 'ANA', 'SANZ', 'anas@somemail.com', '685777999', 0, 1, 2, 0, 0),
('N-MAD-000002', 'RODRIGO', 'SANZ', 'rods@themail.com', '6857888999', 1, 0, 1, 3, 4),
('N-MAD-000003', 'MIGUEL', 'BARRERA', 'migb@othermail.com', '685444777', 0, 1, 1, 0, 0),
('N-MAD-000004', 'SARA', 'PEREZ', 'sarp@somemail.com', '685999555', 1, 0, 5, 0, 0),
('N-MAD-000005', 'JOSE', 'VIDAL', 'josv@somemail.com', '685222333', 0, 1, 0, 0, 0),
('N-MAD-000006', 'PILI', 'CONTRERAS', 'pilc@themail.com', '685111888', 0, 1, 3, 0, 0),
('N-MAD-000007', 'ALBA', 'REYES', 'albr@othermail.com', '685222555', 1, 0, 2, 0, 0)
;
""")

## Doing some manipulations

**Updating** the phone number from "project_people" at row 2.

In [9]:
c.execute("""
UPDATE project_people
SET Phone_num = 685888999
WHERE Employee_id = 'N-MAD-000002'
;
""")

**Creating a TABLE** from a bigger one.

In [10]:
c.execute("""
CREATE TABLE project_managers AS
SELECT F_Name, P_Manager
FROM project_people
WHERE P_Manager = 1
;
""")

**JOINING** from two different ones.

`Nota: La primera línea (CREATE TABLE...) la agregé para agrupar el resultado en una "entidad visible" desde la web SQLViewer, con solo arrastrar hasta la página el archivo de la base de datos (Samsung_Tecs_Projects_MAD.sqlite3) y buscar la tabla creada.`

In [11]:
c.execute("""
CREATE TABLE project_status AS
SELECT 
    pi.Project_id, pi.Project_Name,
    pp.Phase_name
FROM project_identification AS pi
JOIN project_phases AS pp
ON pi.Phase_id = pp.Phase_id
;
""")

**Using LIKE** to see all the projects that match the criteria (Fibra óptica).

`Nota: Igual que el caso anterior, (CREATE TABLE...) <-- mismo objetivo.`

In [12]:
c.execute("""
CREATE TABLE proyectos_de_fibra AS
SELECT *
FROM project_identification
WHERE Project_Name LIKE '%Fibra óptica%'
;
""")

**ADDING** a column to a table. **DDL command** or Data Definition Language command. It means that works on the structure of the table and not on the data contained in the table.

In [13]:
c.execute("""
ALTER TABLE project_identification
ADD Country TEXT
;
""")

In [14]:
c.execute("""
UPDATE project_identification
SET Country = 'Spain'
;
""")

### Let's commit !

In [15]:
conn.commit()

**TRUNCATING / DELETING / DROPPING**
La principal diferencia entre estos comandos está en que:

* TRUNCATE es un comando DDL por lo que elimina las filas de la tabla, más no la existencia de la tabla.
* DELETING es un comando DML por lo que actúa sobre los datos de la tabla y no sobre su estructura, se puede combinar con WHERE para filtrar/definir sobre qué actuar.
* DROP es un comando DDL que elimina tanto los datos como la columna/tabla de la tabla/base de datos.

In [16]:
c.execute("""
DROP TABLE proyectos_de_fibra
;
""")

**Nota:** Al finalizar se debe cerrar el cursor y la conexión.

In [17]:
c.close()
conn.commit()
conn.close()